In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [2]:
df = pd.read_csv("sim_data_full.csv")
df_pfpr = df[df["baseline_transmission_metric"]=="pfpr"].reset_index(drop=True)
df_pfpr = df_pfpr[df_pfpr["transmission_level"] > 0.01].reset_index(drop=True)

In [3]:
df_pfpr["cost_per_1k"] = df_pfpr["cost_mean"]/5
df_pfpr["percent_cases_averted"] = df_pfpr["frac_averted_clinical__all"]*100

In [4]:
scenarios_to_plot = [1,4,13,16,30]
sahel_esmc_scenarios = [35,36] # 35,36 only for e-SMC in Sahel
itn_scenario = [31]

In [5]:
df_scatterplot = df_pfpr[np.in1d(df_pfpr["scenario_number"], scenarios_to_plot)]
df_sahel_extra = df_pfpr[np.logical_and(df_pfpr["archetype"]=="Sahel", 
                                        np.in1d(df_pfpr["scenario_number"], sahel_esmc_scenarios))]
df_scatterplot_itn = df_pfpr[np.in1d(df_pfpr["scenario_number"], itn_scenario)]
# df_list = []
# for a, sdf in df_pfpr.groupby("archetype"):
#     df_list.append(sdf[np.in1d(sdf["scenario_number"], scenarios_to_plot)])
#     if a == "Sahel":
#         df_list.append(sdf[np.in1d(sdf["scenario_number"], sahel_esmc_scenarios)])
                       
# df_scatterplot = pd.concat(df_list, ignore_index=True)

In [6]:
markersize=80

for a, sdf in df_scatterplot.groupby(["archetype", "transmission_level"]):
    plt.figure(dpi=300)
    sdf_without_no_iptsc = sdf[np.logical_not(sdf["scenario_number"]==30)]
    if a[0] == "Sahel":
        palette = ["#5292C2","#F48420"]
    else:
        palette = ["#009F89","#F48420"]
    sns.scatterplot(data=sdf_without_no_iptsc, x="percent_cases_averted", y="cost_per_1k", style="interval", hue="drug_type", size="interval", 
                    size_order=["month","term"], zorder=3, markers=["o","s"],palette=palette,sizes=(markersize*1.5,markersize*4))

    no_iptsc = sdf[sdf["scenario_number"]==30]
    plt.axhline(no_iptsc["cost_per_1k"].iloc[0], linestyle='dotted', color='gray', zorder=0)
    # plt.axhspan(no_iptsc["cost_low"].iloc[0]/5, no_iptsc["cost_high"].iloc[0]/5,
    #             alpha=0.2, color='gray', zorder=0)
    
    itn_sdf = df_scatterplot_itn[np.logical_and(df_scatterplot_itn["transmission_level"]==a[1],
                                                df_scatterplot_itn["archetype"]==a[0])]
    yerr = np.zeros([2,len(itn_sdf)])
    yerr[0:] = itn_sdf["cost_mean"]-itn_sdf["cost_low"]
    yerr[1:] = itn_sdf["cost_high"]-itn_sdf["cost_mean"]
    yerr /= 5.
    plt.errorbar(itn_sdf["percent_cases_averted"], 
         itn_sdf["cost_per_1k"], 
         # xerr=itn_sdf["frac_averted_clinical__all_STDEV"]*100,
         # yerr=yerr,
        linestyle="none", marker="*",
        color="black", zorder=1, markersize=markersize/5)

    yerr = np.zeros([2,len(sdf_without_no_iptsc)])
    yerr[0:] = sdf_without_no_iptsc["cost_mean"]-sdf_without_no_iptsc["cost_low_IPTsc_only"]
    yerr[1:] = sdf_without_no_iptsc["cost_high_IPTsc_only"]-sdf_without_no_iptsc["cost_mean"]
    # yerr[0:] = sdf_without_no_iptsc["cost_mean"]-sdf_without_no_iptsc["cost_low"]
    # yerr[1:] = sdf_without_no_iptsc["cost_high"]-sdf_without_no_iptsc["cost_mean"]
    yerr /= 5.

    plt.errorbar(sdf_without_no_iptsc["percent_cases_averted"], 
                 sdf_without_no_iptsc["cost_per_1k"], 
                 xerr=sdf_without_no_iptsc["frac_averted_clinical__all_STDEV"]*100,
                 yerr=yerr,
                linestyle="none",
                color="black", zorder=2)
    
    if a[0] == "Sahel":
        esmc_sdf = df_sahel_extra[df_sahel_extra["transmission_level"]==a[1]]
        
        
        for s, ssdf in esmc_sdf.groupby("scenario_number"):
            if s == 35:
                smc_color = "#EFAC1F"
            elif s == 36:
                smc_color = "#D51C5B"
            
            yerr = np.zeros([2,len(ssdf)])
            yerr[0:] = ssdf["cost_mean"]-ssdf["cost_low_SMC_only"]
            yerr[1:] = ssdf["cost_high_SMC_only"]-ssdf["cost_mean"]
            yerr /= 5.
            plt.errorbar(ssdf["percent_cases_averted"], 
                 ssdf["cost_per_1k"], 
                 xerr=ssdf["frac_averted_clinical__all_STDEV"]*100,
                 yerr=yerr,
                linestyle="none", marker="D",
                color=smc_color, zorder=1,markersize=markersize/5)
            # yerr = np.zeros([2,len(esmc_sdf)])
            # yerr[0:] = esmc_sdf["cost_mean"]-esmc_sdf["cost_low"]
            # yerr[1:] = esmc_sdf["cost_high"]-esmc_sdf["cost_mean"]
            # yerr /= 5.
            # plt.errorbar(esmc_sdf["percent_cases_averted"], 
            #      esmc_sdf["cost_per_1k"], 
            #      xerr=esmc_sdf["frac_averted_clinical__all_STDEV"]*100,
            #      yerr=yerr,
            #     linestyle="none", marker="D",
            #     color="purple", zorder=1)
    # plt.scatter(sdf["frac_averted_clinical__all"], sdf["cost"]

#     no_iptsc = sdf[sdf["scenario_number"]==30]
#     plt.axhline(no_iptsc["cost_per1kpop_peryear"].iloc[0], color='black', linestyle='dotted')

    plt.title(f"{a[0]} - pfpr {a[1]}") 
    plt.legend(fontsize=10)
    # plt.ylim([0,15000])
    # plt.xlim([0,1])
    plt.xlim([0,85])
    plt.ylim([0,22000])
    ax = plt.gca()
    plt.yticks([0,5000,15000,10000,20000])
    # plt.yticks([0,5000,15000,10000,20000,25000])

    plt.tight_layout()
    plt.savefig(f"figs/{a[0]}_{str(a[1]).zfill(3)}.png")
    plt.savefig(f"figs/{a[0]}_{str(a[1]).zfill(3)}.pdf")
    plt.close("all")

meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped


In [7]:
sdf_without_no_iptsc

,archetype,baseline_transmission_metric,transmission_level,scenario_number,iptsc_rdts_used,iptsc_drugs_used,cases_treated,severe_cases_treated,received_smc,received_ivermectin,...,cost_SMC_mean,cost_mean,cost_low_IPTsc_only,cost_high_IPTsc_only,cost_low_SMC_only,cost_high_SMC_only,cost_low,cost_high,cost_per_1k,percent_cases_averted
909,Southern,pfpr,0.4,1,0.0,5255.2,7895.1,0.0,0.0,0.0,...,0.0,69148.963000,67441.02300,70856.903,69148.963000,69148.963000,38229.15300,100068.773,13829.792600,7.852370
912,Southern,pfpr,0.4,4,0.0,20909.9,6331.2,0.0,0.0,0.0,...,0.0,71741.809375,64240.38275,79243.236,71741.809375,71741.809375,40814.94275,102668.676,14348.361875,30.065877
921,Southern,pfpr,0.4,13,0.0,5258.9,6659.5,0.0,0.0,0.0,...,0.0,69065.964500,67356.82200,70775.107,69065.964500,69065.964500,42716.67200,95415.257,13813.192900,25.499282
924,Southern,pfpr,0.4,16,0.0,20898.8,5766.6,0.0,0.0,0.0,...,0.0,102025.629500,94528.18500,109523.074,102025.629500,102025.629500,73191.76500,130859.494,20405.125900,39.706034


In [8]:
sdf[["

SyntaxError: EOL while scanning string literal (Temp/ipykernel_14248/1678201030.py, line 1)